In [4]:
pip install -qU langchain-text-splitters langchain-community langgraph langchain-huggingface faiss-cpu bitsandbytes accelerate pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install olefile

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [7]:
files_path = "../data/raw/files"
csv_path = "../data/raw/data_list.csv"

In [8]:
import pandas as pd
import os

# CSV 읽기
df = pd.read_csv(csv_path)

# 파일 존재 확인
df['파일 경로'] = df['파일명'].apply(lambda x: os.path.join(files_path, x))
df = df[df['파일 경로'].apply(os.path.exists)]
print(df.head())

         공고 번호  공고 차수                                       사업명        사업 금액  \
0  20241001798    0.0  한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화  130000000.0   
1  20241002912    0.0        2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선  129300000.0   
2  20240827859    0.0                EIP3.0 고압가스 안전관리 시스템 구축 용역   40000000.0   
3  20240430918    0.0                      도시계획위원회 통합관리시스템 구축용역  150000000.0   
4  20240430896    0.0              봉화군 재난통합관리시스템 고도화 사업(협상)(긴급)  900000000.0   

       발주 기관                공개 일자            입찰 참여 시작일            입찰 참여 마감일  \
0       한영대학  2024-10-04 13:51:23                  NaN  2024-10-15 17:00:00   
1     한국연구재단  2024-10-04 15:01:52  2024-10-14 10:00:00  2024-10-16 14:00:00   
2  한국생산기술연구원  2024-08-28 11:31:02  2024-08-29 09:00:00  2024-09-09 10:00:00   
3      인천광역시  2024-04-18 16:26:32  2024-05-02 10:00:00  2024-05-09 16:00:00   
4   경상북도 봉화군  2024-04-18 16:33:28  2024-04-26 09:00:00  2024-04-30 17:00:00   

                                            

In [10]:
# hwp 텍스트 추출
import olefile
import zlib
import struct
#### 추가 ####
import re
import unicodedata

class HWPExtractor(object):
    FILE_HEADER_SECTION = "FileHeader"
    HWP_SUMMARY_SECTION = "\x05HwpSummaryInformation"
    SECTION_NAME_LENGTH = len("Section")
    BODYTEXT_SECTION = "BodyText"
    HWP_TEXT_TAGS = [67]

    def __init__(self, filename):
        self._ole = self.load(filename)
        self._dirs = self._ole.listdir()

        self._valid = self.is_valid(self._dirs)
        if (self._valid == False):
            raise Exception("Not Valid HwpFile")

        self._compressed = self.is_compressed(self._ole)
        self.text = self._get_text()

    # 파일 불러오기
    def load(self, filename):
        return olefile.OleFileIO(filename)

    # hwp 파일인지 확인 header가 없으면 hwp가 아닌 것으로 판단하여 진행 안함
    def is_valid(self, dirs):
        if [self.FILE_HEADER_SECTION] not in dirs:
            return False

        return [self.HWP_SUMMARY_SECTION] in dirs

	# 문서 포맷 압축 여부를 확인
    def is_compressed(self, ole):
        header = self._ole.openstream("FileHeader")
        header_data = header.read()
        return (header_data[36] & 1) == 1

	# bodytext의 section들 목록을 저장
    def get_body_sections(self, dirs):
        m = []
        for d in dirs:
            if d[0] == self.BODYTEXT_SECTION:
                m.append(int(d[1][self.SECTION_NAME_LENGTH:]))

        return ["BodyText/Section"+str(x) for x in sorted(m)]

    # text를 뽑아내는 함수
    def get_text(self):
        return self.text

	# 전체 text 추출
    def _get_text(self):
        sections = self.get_body_sections(self._dirs)
        text = ""
        for section in sections:
            text += self.get_text_from_section(section)
            text += "\n"

        self.text = text
        return self.text

	# section 내 text 추출
    def get_text_from_section(self, section):
        bodytext = self._ole.openstream(section)
        data = bodytext.read()

        unpacked_data = zlib.decompress(data, -15) if self._compressed else data
        size = len(unpacked_data)

        i = 0

        text = ""
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            level = (header >> 10) & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in self.HWP_TEXT_TAGS:
                rec_data = unpacked_data[i+4:i+4+rec_len]

                ############## 정제 추가된 부분 #############
                decode_text = rec_data.decode('utf-16')
                # 문자열을 담기 전 정제하기
                res = remove_non_standard_characters(remove_chinese_characters(decode_text))

                text += res
                text += "\n"

            i += 4 + rec_len

        return text

# 중국어(한자) 제거
def remove_chinese_characters(s: str):
    return re.sub(r'[\u4e00-\u9fff]+', '', s)

# # 바이트 문자열 제거
# def remove_control_characters(s):
#     return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

# 바이트 문자열 및 기타 비표준 문자 제거를 위한 보강된 함수
def remove_non_standard_characters(s: str) -> str:
    """
    유니코드 범주 C(제어문자 등)와 기타 비표준 문자를 제거합니다.
    """
    # 유니코드 C (Other) 범주에 속하는 모든 문자를 1차로 제거
    s = "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")
    
    # 영문, 숫자, 한글, 공백, 구두점을 제외한 모든 문자를 2차로 제거
    # \p{P} 대신 가장 흔히 사용되는 구두점 집합을 직접 명시하여 호환성 강화
    # 쉼표, 마침표, 물음표, 느낌표 등
    s = re.sub(r'[^a-zA-Z0-9가-힣\s\.,?!:;\'"()\[\]{}<>/~`@#$%^&*+-=|\\_]+', '', s)
    
    return s


# text 추출 함수 
def get_hwp_text(filename):
    hwp = HWPExtractor(filename)
    return hwp.get_text()


import os

def process_hwp_folder(folder_path):
    """
    주어진 폴더 내의 모든 HWP 파일에서 텍스트를 추출하여 딕셔너리로 반환합니다.

    Args:
        folder_path (str): HWP 파일들이 있는 폴더의 경로.

    Returns:
        dict: 파일 이름을 키로, 추출된 텍스트를 값으로 가지는 딕셔너리.
              HWP 파일이 아닌 경우 해당 파일은 결과에서 제외됩니다.
    """
    all_hwp_texts = {}

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".hwp"):
            filepath = os.path.join(folder_path, filename)
            try:
                print(f"Processing: {filename}")
                hwp_text = get_hwp_text(filepath)
                all_hwp_texts[filename] = hwp_text
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return all_hwp_texts

# --- 사용 예시 ---
# HWP 파일들이 있는 폴더 경로를 지정하세요.
# 예: 'C:/Users/YourUsername/Documents/HWP_Files' 또는 './HWP_Documents'
hwp_folder_path = files_path 

# 함수 실행
hwp_extracted_texts = process_hwp_folder(hwp_folder_path)

Processing: 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp
Processing: 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp
Processing: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp
Processing: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp
Processing: 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.hwp
Processing: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
Processing: 한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.hwp
Processing: 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.hwp
Processing: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp
Processing: 부산관광공사_경영정보시스템 기능개선.hwp
Processing: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.hwp
Processing: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.hwp
Processing: 한국재정정보원_e나라도움 업무시스템 웹 접근성 컨설팅.hwp
Processing: 한국생산기술연구원_2세대 전자조달시스템  기반구축사업.hwp
Processing: 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp
Processing: 한국지식재산보호원_IP-NAVI  해외지식재산센터 사업관리 시스템 기능개.hwp
Processing: 대한상공회의소_기업 재생에너지 지원센터 홈페이지 개편 및 시스템 고.hwp
Processing: 인천광역시 동구_수도국산달동네박물관 전시해설 시스템 구축(협상에 .hwp
Processing: 대한장애인체육회_2025년 전국장애인체육대회 전산 및 시스템, 홈페이지 .hwp

In [11]:
# 결과 확인 (예: 첫 번째 파일의 텍스트만 출력)

# --- HWP 샘플 확인 ---
print("=== HWP 샘플 ===")
if hwp_extracted_texts:
    first_file = list(hwp_extracted_texts.keys())[73]
    print(f"\n--- Text from {first_file} ---")
    print(hwp_extracted_texts[first_file][:3000] + "...") # 첫 3000자만 출력
else:
    print("No HWP files found or processed.")

=== HWP 샘플 ===

--- Text from (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.hwp ---


제 안 요 청 서

사 업 명
통합 정보시스템 구축 사전 컨설팅
주관기관
재단법인 예술경영지원센터
2024년 04월


구 분
소 속
전화번호
 이메일
입찰관련
경영지원팀
02-708-2212
ksj37@gokams.or.kr
사업관련(한국 미술시장 정보시스템)
시각정보팀
02-2098-2946
shwan@gokams.or.kr

 . 사업개요


1
 사업일반
 사 업 명 : 통합 정보시스템 구축 사전 컨설팅
 사업기간 : 계약체결일로부터 ~ 2024년 11월 29일
 사업예산 : 50,000,000(금 오천만원/VAT포함)
 사업추진방식 : 제한경쟁입찰(협상에 의한 계약)

2
 추진목적
  미술진흥법 제23조에 의거 통합미술정보시스템 구축운영
  통합미술정보시스템으로의 시스템 개편 및 고도화 전략 수립을 위한 기반 마련 필요성 대두 
  이용자 기반의 통합미술정보시스템 구축 방향 수립을 위한 사전 분석

3
 사업내용
  환경 및 현황분석
  통합미술정보시스템 구축 데이터 분석 및 설계
  통합정보시스템 벤치마킹 사례 조사
  비전 및 전략 수립 
  통합미술정보시스템 서비스 개발

 . 시스템 현황

1
 업무 현황
 현재 시각예술정보시스템 한국미술시장정보시스템을 운영하고 있음

웹사이트 명
주소(URL)
비고
한국 미술시장 정보시스템
http://www.k-artmarket.krh
웹/모바일
 한국 미술시장 정보 시스템(http://www.k-artmarket.krh)
  미술작품 거래정보 제공
   - 국내 경매회사를 통해 거래되는 미술작품 거래정보(작가, 작 품, 가격, 거래 등) 제공
   - 작품 정보, 이미지, 거래 현황 정보 제공



미술작품 거래정보 제공(페이지)
경매별 출품낙찰 작품
 미술품 경매 개최 정보 제공
   - 국내 미술품 경매회사 정보 제공
   - 국내 미술품 경매회사 개최 경매 정보 제공
  미

In [12]:
# pdf 텍스트 추출
import PyPDF2
import re
import unicodedata

# 중국어 제거
def remove_chinese_characters(s: str):
    return re.sub(r'[\u4e00-\u9fff]+', '', s)

# 바이트 문자열 및 기타 비표준 문자 제거를 위한 보강된 함수
def remove_non_standard_characters(s: str) -> str:
    """
    유니코드 범주 C(제어문자 등)와 기타 비표준 문자를 제거합니다.
    """
    # 유니코드 C (Other) 범주에 속하는 모든 문자를 1차로 제거
    s = "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")
    
    # 영문, 숫자, 한글, 공백, 구두점을 제외한 모든 문자를 2차로 제거
    # \p{P} 대신 가장 흔히 사용되는 구두점 집합을 직접 명시하여 호환성 강화
    # 쉼표, 마침표, 물음표, 느낌표 등
    s = re.sub(r'[^a-zA-Z0-9가-힣\s\.,?!:;\'"()\[\]{}<>/~`@#$%^&*+-=|\\_]+', '', s)
    
    return s

class PDFExtractor(object):
    """
    PDF 파일에서 텍스트를 추출하는 클래스
    """
    def __init__(self, filename):
        self.filename = filename
        self.text = self._get_text()

    # text를 뽑아내는 함수
    def get_text(self):
        return self.text
    
    def _get_text(self):
        text = ""
        try:
            with open(self.filename, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                # 페이지별로 텍스트 추출
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        # HWP 코드와 동일한 정제 함수 적용
                        clean_text = remove_non_standard_characters(remove_chinese_characters(page_text))
                        text += clean_text
                        text += "\n"
        except Exception as e:
            print(f"Error processing PDF file: {e}")
            return None
        return text

def get_pdf_text(filename):
    """추출된 텍스트 반환"""
    pdf = PDFExtractor(filename)
    return pdf.get_text()

def process_pdf_folder(folder_path):
    """
    주어진 폴더 내의 모든 pdf 파일에서 텍스트를 추출하여 딕셔너리로 반환합니다.

    Args:
        folder_path (str): pdf 파일들이 있는 폴더의 경로.

    Returns:
        dict: 파일 이름을 키로, 추출된 텍스트를 값으로 가지는 딕셔너리.
              pdf 파일이 아닌 경우 해당 파일은 결과에서 제외됩니다.
    """
    all_pdf_texts = {}

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            filepath = os.path.join(folder_path, filename)
            try:
                print(f"Processing: {filename}")
                pdf_text = get_pdf_text(filepath)
                all_pdf_texts[filename] = pdf_text
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return all_pdf_texts

# --- 사용 예시 ---
# PDF 파일들이 있는 폴더 경로를 지정하세요.
# 예: 'C:/Users/YourUsername/Documents/HWP_Files' 또는 './PDF_Documents'
pdf_folder_path = files_path  

# 함수 실행
pdf_extracted_texts = process_pdf_folder(pdf_folder_path)

        
        
# pip install langchain_community pypdf

# import os
# import re
# import unicodedata
# import asyncio
# from typing import Dict, List
# from langchain_community.document_loaders import PyPDFLoader

# def remove_chinese_characters(s: str) -> str:
#     """중국어 문자를 제거합니다."""
#     return re.sub(r'[\u4e00-\u9fff]+', '', s)

# def remove_control_characters(s: str) -> str:
#     """유니코드 제어 문자를 제거합니다."""
#     return "".join(ch for ch in s if unicodedata.category(ch)[0] != "C")

# def clean_text(text: str) -> str:
#     """
#     강력한 전처리 함수:
#     중국어 문자, 제어 문자 제거 후 연속 공백/줄바꿈을 정리합니다.
#     """
#     text = remove_chinese_characters(text)
#     text = remove_control_characters(text)
#     text = re.sub(r'\n+', '\n', text)
#     text = re.sub(r'[ \t]+', ' ', text)
#     return text.strip()

# async def load_and_clean_pdf_page(filepath: str) -> Dict[str, str]:
#     """
#     langchain의 PyPDFLoader를 사용하여 PDF 파일을 비동기적으로 로드하고 전처리합니다.
#     """
#     print(f"Loading and cleaning: {os.path.basename(filepath)}")
#     try:
#         loader = PyPDFLoader(filepath)
#         all_pages_data = []
        
#         async for page in loader.alazy_load():
#             page_content = page.page_content
#             cleaned_page_content = clean_text(page_content)
#             all_pages_data.append({
#                 "page_number": page.metadata.get("page", -1),
#                 "content": cleaned_page_content
#             })
        
#         full_text = " ".join([data["content"] for data in all_pages_data])
#         return {os.path.basename(filepath): full_text}
        
#     except Exception as e:
#         print(f"Error processing {os.path.basename(filepath)}: {e}")
#         return {}

# async def process_pdf_folder_async(folder_path: str) -> Dict[str, str]:
#     """
#     주어진 폴더 내 모든 PDF 파일을 비동기적으로 처리하여 딕셔너리로 반환합니다.
#     """
#     all_pdf_texts = {}
#     tasks = []

#     for filename in os.listdir(folder_path):
#         if filename.lower().endswith(".pdf"):
#             filepath = os.path.join(folder_path, filename)
#             tasks.append(load_and_clean_pdf_page(filepath))

#     results = await asyncio.gather(*tasks)

#     for result in results:
#         all_pdf_texts.update(result)

#     return all_pdf_texts







# import os
# import re
# from PyPDF2 import PdfReader

# def clean_text(text: str) -> str:
#     """
#     PDF에서 추출한 텍스트를 전처리합니다.
#     """
#     text = text.replace("\n", " ").replace("\t", " ")
#     text = re.sub(r"\s+", " ", text)  # 여러 공백 하나로
#     return text.strip()

# def get_pdf_text(filepath: str) -> str:
#     """
#     주어진 PDF 파일에서 텍스트를 추출합니다.
#     """
#     reader = PdfReader(filepath)
#     text = ""
#     for page in reader.pages:
#         page_text = page.extract_text()
#         if page_text:  # None 방지
#             text += page_text + " "
#     return clean_text(text)

# def process_pdf_folder(folder_path: str) -> dict:
#     """
#     주어진 폴더 내의 모든 PDF 파일에서 텍스트를 추출하여 딕셔너리로 반환합니다.

#     Args:
#         folder_path (str): PDF 파일들이 있는 폴더의 경로.

#     Returns:
#         dict: 파일 이름을 키로, 추출된 텍스트를 값으로 가지는 딕셔너리.
#               PDF 파일이 아닌 경우 해당 파일은 결과에서 제외됩니다.
#     """
#     all_pdf_texts = {}

#     for filename in os.listdir(folder_path):
#         if filename.lower().endswith(".pdf"):
#             filepath = os.path.join(folder_path, filename)
#             try:
#                 print(f"Processing: {filename}")
#                 pdf_text = get_pdf_text(filepath)
#                 all_pdf_texts[filename] = pdf_text
#             except Exception as e:
#                 print(f"Error processing {filename}: {e}")

#     return all_pdf_texts

# pdf_extracted_texts = process_pdf_folder(files_path)




# from langchain_community.document_loaders import PyPDFLoader

# all_pages = []

# # files_path 안에 있는 모든 pdf 파일 반복
# for filename in os.listdir(files_path):
#     if filename.lower().endswith(".pdf"):
#         file_path = os.path.join(files_path, filename)

#         loader = PyPDFLoader(file_path)

#         # 비동기 로딩
#         pages = []
#         async for page in loader.alazy_load():
#             pages.append(page)

#         all_pages.extend(pages)

# print(f"총 {len(all_pages)} 페이지 로드 완료")





# import re
# import unicodedata

# # HWP와 동일한 전처리 함수
# def remove_chinese_characters(s: str):   
#     return re.sub(r'[\u4e00-\u9fff]+', '', s)
    
# def remove_control_characters(s):    
#     return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

# def clean_text(text: str) -> str:
#     text = remove_chinese_characters(text)
#     text = remove_control_characters(text)
#     # 연속 공백/줄바꿈 정리
#     text = re.sub(r'\n+', '\n', text)
#     text = re.sub(r'[ \t]+', ' ', text)
#     return text

# # all_pages 안의 PDF 텍스트 전처리
# clean_pdf_pages = []
# for page in all_pages:
#     page_content = page.page_content
#     page_content = clean_text(page_content)
#     page.page_content = page_content
#     clean_pdf_pages.append(page)

Processing: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
Processing: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
Processing: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
Processing: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf


In [13]:
# 결과 확인 (예: 첫 번째 파일의 텍스트만 출력)

# --- PDF 샘플 확인 ---
print("=== PDF 샘플 ===")
if pdf_extracted_texts:
    first_file = list(pdf_extracted_texts.keys())[0]
    print(f"\n--- Text from {first_file} ---")
    print(pdf_extracted_texts[first_file][:3000] + "...") # 첫 3000자만 출력
else:
    print("No PDF files found or processed.")


# # --- PDF 샘플 확인 ---
# print("\n=== PDF 샘플 ===")
# for i, page in enumerate(clean_pdf_pages[:3]):  # 상위 3페이지만 확인
#     print(f"\n--- 페이지 {i+1} ---")
#     print(page.page_content[:500], "...")

=== PDF 샘플 ===

--- Text from 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf ---
2025년도 중이온가속기용 극저온시스템운전 용역 과업지시서2024. 10.중이온가속기연구소가속기운영부 극저온팀
문서번호 -개정번호 0발 행 일 2024. 10.302025년도 중이온가속기용 극저온시스템 운전 용역 과업지시서문서상태 식별 (다음 중 하나에 체크 ) 문서상태 -1:최종설계 ,해석 ,구매 등에 이용되는 설계 확인된 정보 문서상태 -2:추정기법 등 설계가정을 사용하여 기술된 정보 문서상태 -3:설계확인 및 추정되지 않은 정보소속 및 직위 성 명 서 명작 성극저온팀 /연구위원 유정현검 토품질관리실 /연구위원 신일경극저온팀 /팀장 신재희승 인가속기운영부 /부장 정연세
2025년도 중이온가속기용 극저온시스템 운전 용역 과업지시서문서번호 : -개정번호 : 0발 행 일 : 2024. 10.30페 이 지 : 3/19개정이력개정번호 개정일자 개  정  사  유1 2024. 07.03 최초발행
2025년도 중이온가속기용 극저온시스템 운전 용역 과업지시서문서번호 : -개정번호 : 0발 행 일 : 2024. 10.30페 이 지 : 4/19목       차1.서  론 .................................................................................................................. 52.과업범위 및 일정 .............................................................................................. 63.과업 내용 ............................................................................................................ 64.제출문서 ..................................

In [17]:
import os

# 저장할 폴더 경로
processed_dir = "../data/processed/processed_text"
os.makedirs(processed_dir, exist_ok=True)

def save_texts(texts_dict, prefix=""):
    """
    전처리된 텍스트를 파일별로 저장하는 함수
    
    Args:
        texts_dict (dict): {"파일명": "텍스트"} 형태
        prefix (str): 저장 시 앞에 붙일 접두사 (예: "hwp_", "pdf_")
    """
    for filename, text in texts_dict.items():
        # 확장자 제거
        base_name = os.path.splitext(filename)[0]
        save_path = os.path.join(processed_dir, f"{prefix}{base_name}.txt")
        
        with open(save_path, "w", encoding="utf-8") as f:
            f.write(text if text else "")
        
        print(f"저장 완료: {save_path}")


# HWP 저장
save_texts(hwp_extracted_texts, prefix="hwp_")

# PDF 저장
save_texts(pdf_extracted_texts, prefix="pdf_")

저장 완료: ../data/processed/processed_text/hwp_한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.txt
저장 완료: ../data/processed/processed_text/hwp_케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .txt
저장 완료: ../data/processed/processed_text/hwp_수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.txt
저장 완료: ../data/processed/processed_text/hwp_국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.txt
저장 완료: ../data/processed/processed_text/hwp_조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.txt
저장 완료: ../data/processed/processed_text/hwp_국민연금공단_2024년 이러닝시스템 운영 용역.txt
저장 완료: ../data/processed/processed_text/hwp_한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.txt
저장 완료: ../data/processed/processed_text/hwp_울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.txt
저장 완료: ../data/processed/processed_text/hwp_BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).txt
저장 완료: ../data/processed/processed_text/hwp_부산관광공사_경영정보시스템 기능개선.txt
저장 완료: ../data/processed/processed_text/hwp_국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.txt
저장 완료: ../data/processed/processed_text/hwp_고양도시관리공사_관산근린공원 다목적구장 

In [15]:
# hwp 청킹

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Document 객체 리스트로 변환
hwp_docs = [
    Document(page_content=text, metadata={"source": filename})
    for filename, text in hwp_extracted_texts.items()
]

# 청킹
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

hwp_splits = splitter.split_documents(hwp_docs)

print(f"총 {len(hwp_splits)} 청크 생성 완료")
print(hwp_splits[0])  # 첫 번째 청크 확인

총 18903 청크 생성 완료
page_content='제 안 요 청 서
[2024년 기초학문자료센터 시스템 운영 및 
연구성과물 DB구축사업]

2024. 7.
  
담 당
부  서  명
성  명
전    화
사 업
인문사회학술지원팀
최우용
042-869-6722
계 약
운영지원팀
이현우
042-869-6232


이 자료는 한국연구재단 제안서 작성 이외의 목적으로 복제, 전달 및 사용을 금함
목 차' metadata={'source': '한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp'}


In [16]:
# Document 객체 리스트로 변환
pdf_docs = [
    Document(page_content=text, metadata={"source": filename})
    for filename, text in pdf_extracted_texts.items()
]

# 청킹
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

pdf_splits = splitter.split_documents(pdf_docs)

print(f"총 {len(pdf_splits)} 청크 생성 완료")
print(pdf_splits[0])  # 첫 번째 청크 확인


# # pdf 청킹

# from langchain_text_splitters import RecursiveCharacterTextSplitter

# # 청킹 도구 설정
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,      # 한 덩어리 최대 길이
#     chunk_overlap=100,   # 겹치는 부분
#     length_function=len, # 길이 계산 기준
# )

# # 페이지들을 청킹
# all_splits = text_splitter.split_documents(pdf_extracted_texts)

# print(f"총 {len(all_splits)} 개의 청크 생성 완료")
# print(all_splits[0])  # 첫 번째 청크 확인

총 1382 청크 생성 완료
page_content='2025년도 중이온가속기용 극저온시스템운전 용역 과업지시서2024. 10.중이온가속기연구소가속기운영부 극저온팀
문서번호 -개정번호 0발 행 일 2024. 10.302025년도 중이온가속기용 극저온시스템 운전 용역 과업지시서문서상태 식별 (다음 중 하나에 체크 ) 문서상태 -1:최종설계 ,해석 ,구매 등에 이용되는 설계 확인된 정보 문서상태 -2:추정기법 등 설계가정을 사용하여 기술된 정보 문서상태 -3:설계확인 및 추정되지 않은 정보소속 및 직위 성 명 서 명작 성극저온팀 /연구위원 유정현검 토품질관리실 /연구위원 신일경극저온팀 /팀장 신재희승 인가속기운영부 /부장 정연세
2025년도 중이온가속기용 극저온시스템 운전 용역 과업지시서문서번호 : -개정번호 : 0발 행 일 : 2024. 10.30페 이 지 : 3/19개정이력개정번호 개정일자 개  정  사  유1 2024. 07.03 최초발행' metadata={'source': '기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf'}
